In [0]:
from pyspark.sql import SparkSession
import logging
import sys
import os

sys.path.append(os.path.join(os.path.dirname(os.path.abspath('')), 'src'))

from utils.data_validation import valida_fonte, ler_fonte, valida_dados
from utils.metadata_handler import adiciona_metadados
from utils.table_operations import cria_tabela
from config.pipeline_config import configura_bronze_creators, configura_silver_creators

## Camada bronze

In [0]:
bronze_config = configura_bronze_creators()
filename = bronze_config["source_filename"].split('.')[0]

logging.basicConfig(level=getattr(logging, bronze_config["log_level"]))
logger = logging.getLogger(__name__)

print(f"Pipeline da camada bronze iniciado em {bronze_config['processing_timestamp']}")
print(f"Fonte: {bronze_config['source']}")

Pipeline da camada bronze iniciado em 2025-08-19 23:23:14.621271
Fonte: /Volumes/workspace/default/dataeng_raw/wiki_pages.json.gz


In [0]:
try:
    if not valida_fonte(spark, bronze_config):
        raise FileNotFoundError(f"Fonte inacessível: {bronze_config['source']}")

    bronze_df = ler_fonte(spark, bronze_config)
    bronze_config = valida_dados(bronze_df, bronze_config)
    bronze_df = adiciona_metadados(bronze_df, bronze_config)
    
    success = cria_tabela(spark, bronze_df, bronze_config)
    if success:
        print("Camada bronze criada com sucesso!")
    else:
        raise Exception("Erro na criação da tabela bronze")
        
except Exception as e:
    print(f"Erro ao processar camada bronze: {str(e)}")
    raise

INFO:utils.data_validation:Fonte validada: /Volumes/workspace/default/dataeng_raw/wiki_pages.json.gz
INFO:utils.table_operations:Iniciada a criação da tabela delta na camada bronze: workspace.default.bronze_creators_scrape_wiki
INFO:utils.table_operations:Metadados da tabela - Camada: bronze, Salvo em: workspace.default.bronze_creators_scrape_wiki
INFO:utils.table_operations:Tabela criada em 2025-08-19T23:23:14.621271
INFO:utils.table_operations:Linhas: 10
INFO:utils.table_operations:Colunas: 1
INFO:utils.table_operations:Batch ID: b604e6b8
INFO:utils.table_operations:Contém dados: True
INFO:utils.table_operations:Tabela criada com sucesso: workspace.default.bronze_creators_scrape_wiki
INFO:utils.table_operations:Comentário com metadados adicionado com sucesso


Camada bronze criada com sucesso!


## Camada silver

In [0]:
silver_config = configura_silver_creators()

logging.basicConfig(level=getattr(logging, silver_config["log_level"]))
logger = logging.getLogger(__name__)

print(f"Pipeline da camada silver iniciado em {silver_config['processing_timestamp']}")
print(f"Fonte: {silver_config['source']}")

Pipeline da camada silver iniciado em 2025-08-19 23:27:38.467454
Fonte: workspace.default.bronze_creators_scrape_wiki


In [0]:
try:
    if not valida_fonte(spark, silver_config):
        raise FileNotFoundError(f"Fonte inacessível: {silver_config['source']}")

    silver_df = ler_fonte(spark, silver_config)
    silver_config = valida_dados(silver_df, silver_config)
    silver_df = adiciona_metadados(silver_df, silver_config)
    
    success = cria_tabela(spark, silver_df, silver_config)
    if success:
        print("Camada silver criada com sucesso!")
    else:
        raise Exception("Erro na criação da tabela silver")
        
except Exception as e:
    print(f"Erro ao processar camada silver: {str(e)}")
    raise

INFO:utils.data_validation:Fonte validada: workspace.default.bronze_creators_scrape_wiki
INFO:utils.table_operations:Iniciada a criação da tabela delta na camada silver: workspace.default.silver_creators_scrape_wiki
INFO:utils.table_operations:Metadados da tabela - Camada: silver, Salvo em: workspace.default.silver_creators_scrape_wiki
INFO:utils.table_operations:Tabela criada em 2025-08-19T23:27:38.467454
INFO:utils.table_operations:Linhas: 10
INFO:utils.table_operations:Colunas (exceto metadados): 1
INFO:utils.table_operations:Contém dados: True
INFO:utils.table_operations:Percentual de completude: 100.0%
INFO:utils.table_operations:Problemas de qualidade: []
INFO:utils.table_operations:Qualidade atestada: True
INFO:utils.table_operations:Tabela criada com sucesso: workspace.default.silver_creators_scrape_wiki
INFO:utils.table_operations:Comentário com metadados adicionado com sucesso


Camada silver criada com sucesso!
